In [1]:
"""Package Loading"""
"""General"""
from webscraping_dictionaries import *
from webscraping_functions import *
import time
from dateutil import relativedelta
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By  
import requests
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import ssl
import os
#import dotenv
"""Chrome Options"""
#from selenium.webdriver import Chrome 
#from selenium.webdriver.chrome.service import Service 
#from webdriver_manager.chrome import ChromeDriverManager

"""Firefox Options"""
#from selenium.webdriver import Firefox
#from selenium.webdriver.firefox.service import Service
#from webdriver_manager.firefox import GeckoDriverManager


"""Initialize the driver"""

"""Driver Setup"""

#following steps from here: https://www.zenrows.com/blog/scraping-javascript-rendered-web-pages#installing-the-requirements
#critical help using the click method from here https://www.geeksforgeeks.org/click-element-method-selenium-python/

"""Chrome Version"""
#options = webdriver.ChromeOptions()
#options.headless = True
#options.page_load_strategy = 'none' 
#chrome_path = ChromeDriverManager().install()
#chrome_service = Service(chrome_path)
#driver = Chrome(options=options, service=chrome_service) 
#driver.implicitly_wait(5)

"""Firefox Version"""  
#options = webdriver.FirefoxOptions()
#options.headless = True
#options.add_argument("--headless=new")
#options.page_load_strategy = 'none' 
#firefox_path = GeckoDriverManager().install() 
#firefox_service = Service(firefox_path)
#driver = Firefox(options=options, service=firefox_service)
#driver.implicitly_wait(5)

[WDM] - Downloading: 19.9kB [00:00, 19.7MB/s]                   
[WDM] - Downloading: 100%|██████████| 1.75M/1.75M [00:01<00:00, 1.19MB/s]


'Firefox Version'

"""DEBUGGING SANDBOX"""

In [ ]:
#table this for now, since we still want to check for comp plan and ordinance keywords anyway
"Charlotte County Solar Development Page"
def charlotte_county_solar_development():
    driver.get("https://www.charlottecountyva.gov/departments/planning___zoning/solar_development.php")
    time.sleep(1)
    table_rows = driver.find_elements(By.CSS_SELECTOR,"tr")
    

In [ ]:
#come back to this another day
all_dictionaries=[agendacenter_dictionary,boarddocs_dictionary,civicclerk_dictionary,civicweb_dictionary,document_center_dictionary,escribe_dictionary,event_list_dictionary,granicus_dictionary,granicus_2_dictionary,laserfiche_dictionary,legistar_dictionary,meetingstable_dictionary,novusagenda_dictionary,php_table_dictionary,primegov_dictionary,county_dictionary_single_variable,county_dictionary_double_variable,city_dictionary_single_variable,city_dictionary_double_variable]

for dictionary in all_dictionaries:
    for entry in dictionary:
        print(entry)

In [67]:
"""Nottoway County"""
def nottoway_county(url, governing_body):
    driver.get(url)
    time.sleep(2)
    messages = []
    years = driver.find_elements(By.CSS_SELECTOR,"h3[class*='agenda-toggle'")
    current_year = [item for item in years if str(datetime.date(datetime.now()).year) in item.text]
    current_year[0].click()
    table_rows = driver.find_elements(By.CSS_SELECTOR,"tr")
    meetings = [item for item in table_rows if item.text != ""]
    future_meetings = [item for item in meetings if check_meeting_date(item.text)==True]
    agenda_links=[item.find_element(By.CSS_SELECTOR,"a[href*=Agenda").get_attribute("href") for item in future_meetings if 'Agenda' in item.text]
    for item in agenda_links:
        driver.get(item)
        time.sleep(2)
        agenda_content = driver.find_elements(By.CSS_SELECTOR,"div[class*=textLayer")
        readable = check_readability(agenda_content)
        if readable==True:
            agenda_search = search_agenda_for_keywords(agenda_content)
            if agenda_search != []:
                messages.append("Keyword(s) " + ", ".join(agenda_search) + " found in upcoming meeting for Nottoway County " + governing_body + ". " + item)
        elif readable==False:
            messages.append("New meeting document available for Nottoway County " + governing_body +". Cannot scan for keywords. " + item)
    return messages

In [5]:
driver.get("https://nottoway.org/administration/boards___commissions/agenda_packets_minutes.php")

In [6]:
years = driver.find_elements(By.CSS_SELECTOR,"h3[class*='agenda-toggle'")

In [7]:
current_year = [item for item in years if str(datetime.date(datetime.now()).year) in item.text]

In [9]:
current_year[0].click()

In [10]:
table_rows = driver.find_elements(By.CSS_SELECTOR,"tr")

In [20]:
table_rows[1].find_element()

'05/13/2024\nMay 21, 2024 Planning Commission Agenda and Packet\nAgenda\nPacket'

In [4]:
nottoway_county(county_dictionary_double_variable["Nottoway County Planning Commission"][1],county_dictionary_double_variable["Nottoway County Planning Commission"][2])

ParserError: String does not contain a date: 

In [205]:
"""Henrico County"""
def henrico_county_bos(url):
    driver.get(url)
    time.sleep(1)
    messages = []
    #ignore the first entry in the table rows since it's just the column headers
    table_rows = driver.find_elements(By.CSS_SELECTOR,'tr')[1:]
    latest_meeting = table_rows[0]
    future_meeting = check_meeting_date(latest_meeting.text)
    if future_meeting == True:
        meeting_url = latest_meeting.find_element(By.CSS_SELECTOR,"a").get_attribute("href")
        meeting_title = latest_meeting.text.split("Meeting")[0]
        driver.get(meeting_url)
        time.sleep(2)
        agenda_content = driver.find_elements(By.CSS_SELECTOR,"div[class*=textLayer")
        agenda_search = search_agenda_for_keywords(agenda_content)
        if agenda_search != []:
            messages.append("Keyword(s) " + ", ".join(agenda_search) + " found in upcoming meeting for Henrico County Board of Supervisors in " + meeting_title + ". " + meeting_url)
    return messages 